In [ ]:
import json
import os

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from pycocotools.coco import COCO

%load_ext autoreload
%autoreload 2
!nvidia-smi

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

In [ ]:
# load relevant file paths for left & right frames and bbox predictions

data_directory_base = '/root/alok/data/'
stereo_frame_pairs_directory = os.path.join(data_directory_base, 'stereo_frame_pairs')

left_image_file_paths, right_image_file_paths = [], []
for directory_name in os.listdir(stereo_frame_pairs_directory):
    directory_path = os.path.join(stereo_frame_pairs_directory, directory_name)
    left_image_file_path = os.path.join(directory_path, 'input', 'left_frame.jpg')
    right_image_file_path = os.path.join(directory_path, 'input', 'right_frame.jpg')
    left_image_file_paths.append(left_image_file_path)
    right_image_file_paths.append(right_image_file_path)
    
    

In [ ]:
# get biomass estimates across all stereo frame pairs
for left_image_file_path, right_image_file_path in zip(left_image_file_paths, right_image_file_paths):
    left_image_fish_detections = get_fish_detections(left_image_file_path)
    right_image_fish_detections = get_fish_detections(right_image_file_path)
    left_right_matches = get_left_right_matches(left_image_fish_detections, right_image_fish_detections)
    for left_right_match in left_right_matches:
        biomass = estimate_biomass(left_right_match)

<h1> MaskRCNN Forward Pass </h1>

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import keras

sys.path.insert(0, '/root/alok/repos/cv_research/lib/maskrcnn/')
from mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.visualize as visualize
from mrcnn.model import log
import mcoco.coco as coco
# import mextra.utils as extra_utils
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline
%config IPCompleter.greedy=True
BASE_DIR = '/root/data/models/erko/mask_rcnn_instance_segmentation'
DATA_DIR = '/root/data/erko/'
WEIGHTS_DIR = os.path.join(BASE_DIR, "weights")
MODEL_DIR = os.path.join(BASE_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(WEIGHTS_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
image_size = 1024
rpn_anchor_template = (1, 2, 4, 8, 16) # anchor sizes in pixels
rpn_anchor_scales = tuple(i * (image_size // 16) for i in rpn_anchor_template)

class FishConfig(Config):
    """Configuration for training on the shapes dataset.
    """
    # name your experiments here
    NAME = "full"

    # Train on 1 GPU and 2 images per GPU. Put multiple images on each
    # GPU if the images are small. Batch size is 2 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 7 + 1  # background + 3 shapes (triangles, circles, and squares)

    # Use smaller images for faster training. 
    IMAGE_MAX_DIM = image_size
    IMAGE_MIN_DIM = image_size
    
    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = rpn_anchor_scales

    # Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    STEPS_PER_EPOCH = 1000

    VALIDATION_STEPS = 300

class InferenceConfig(FishConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    

config = FishConfig()
config.display()

In [ ]:
model_path = '/root/data/models/small_pen/body_parts/logs/body_parts_uncomlpete_1024_20181010_21H08/mask_rcnn_body_parts_uncomlpete_1024_0100.h5'

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"

# model_path = '/root/data/models/erko/mask_rcnn_instance_segmentation/logs/full_20181002_19H09/mask_rcnn_full_0097.h5'
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)


